# Web Scraping: The Stockmarketwire

In [47]:
from datetime import datetime
import numpy as np
import pandas as pd


from bs4 import BeautifulSoup
from selenium import webdriver
import time

import requests
import re



### Obtain list of news from the coverpage

URL definition:

In [48]:
# url definition
url = "http://www.stockmarketwire.com/market-news/"

List of news:

In [97]:
# Request
r1 = requests.get(url)
print(r1.status_code)

# We'll save in coverpage the cover page content
coverpage = r1.content

# Soup creation
soup1 = BeautifulSoup(coverpage, 'html5lib')

# News identification
coverpage_news = soup1.find_all('li', class_="ui-helper-clearfix")
len(coverpage_news)

200


50

Now we have a list in which every element is a news article:

In [98]:
coverpage_news[0]

<li class="ui-helper-clearfix">
		<div class="date">
												02 Oct<br/>15:02
									</div>
		<div class="body">
			<a href="/article/6615311/Katoro-Gold-raises-capital-to-advance-Haneti-project-in-Tanzania.html">Katoro Gold raises capital to advance Haneti project in Tanzania</a>
			<p>Tanzania-focused Katoro Gold said it had raised £0.1m via a share placement to continue to advance its Haneti project in the African nation.

New shares in the company were issed at 1p each.


At...</p>
		</div>
	</li>

In [99]:
n=0
link = coverpage_news[n].find('a')['href']
title = coverpage_news[n].get_text()
article = requests.get(url+link)
article_content = article.content
soup_article = BeautifulSoup(article_content, 'html5lib')



In [100]:
title

'\n\t\t\n\t\t\t\t\t\t\t\t\t\t\t\t02 Oct15:02\n\t\t\t\t\t\t\t\t\t\n\t\t\n\t\t\tKatoro Gold raises capital to advance Haneti project in Tanzania\n\t\t\tTanzania-focused Katoro Gold said it had raised £0.1m via a share placement to continue to advance its Haneti project in the African nation.\n\nNew shares in the company were issed at 1p each.\n\n\nAt...\n\t\t\n\t'

In [80]:
body = soup_article.find_all('p')

In [81]:
body

[<p>Tanzania-focused Katoro Gold said it had raised £0.1m via a share placement to continue to advance its Haneti project in the African nation.
 
 New shares in the company were issed at 1p each.
 
 
 At...</p>,
 <p>US non-farm private-sector employment rose by 135k in September, according to ADP Research Institute.
 
 The rise was broadly in line with expectations for 140k positions.
  
 Story provided by...</p>,
 <p>Insurance investor Randall &amp; Quilter said it had appointed former KPMG partner Mike Walker to the newly-created role of chief restructuring officer.
 
 The role would involve Walker being...</p>,
 <p>Brazil-focused nickel miner Horizonte Minerals said it had appointed Pedro Rodrigues dos Reis as project director of its Araguaia ferronickel project.
 
 Rodrigues dos Reis most recently worked at...</p>,
 <p>Online fashion retailer Asos said it had added four new directors to its board, to replace two departing directors and reflect the company's increasing size.
 
 The

In [82]:
len(body)

53

In [83]:
x = soup_article.find_all('p')

In [84]:
len(x)

53

In [86]:
x[0].get_text()

'Tanzania-focused Katoro Gold said it had raised £0.1m via a share placement to continue to advance its Haneti project in the African nation.\n\nNew shares in the company were issed at 1p each.\n\n\nAt...'

In [60]:
# options = webdriver.ChromeOptions() 
# options.add_experimental_option("excludeSwitches", ['enable-automation'])
# def Article(url):
#     browser = webdriver.Chrome(chrome_options=options, executable_path=r"/usr/bin/chromedriver")
#     browser.get(url)
#     time.sleep(5) # wait for the page to load javascript
#     pageSource = browser.page_source
#     browser.quit()
#     return pageSource

### Let's extract the text from the articles:

First, we'll define the number of articles we want:

In [87]:
number_of_articles = len(coverpage_news)-1
now = datetime.now()

In [88]:
# Empty lists for content, links and titles
news_contents = []
list_links = []
list_titles = []
scrapedTime=[]
postedDate=[]
for n in np.arange(0, number_of_articles):
        
    # Getting the link of the article
    link = coverpage_news[n].find('a')['href']
    print (link)
    list_links.append(link)
    posteddate=re.findall(r'/(\d{4})/(\d{1,2})/(\d{1,2})/', link)
    #posteddate=str(posteddate)
    if posteddate:
        posteddate=','.join(posteddate[0]) 
        posteddate=posteddate.replace(',','/')
    print (posteddate)
    postedDate.append(posteddate)
    
    # Getting the title
    title = coverpage_news[n].get_text()
    list_titles.append(title)
    
    
    # Reading the content (it is divided in paragraphs)
    article = requests.get(url+link)
    article_content = article.content
    soup_article = BeautifulSoup(article_content, 'html5lib')
    x = soup_article.find_all('p')
    scraped = now.strftime("%d/%m/%Y %H:%M:%S")
    scrapedTime.append(scraped)
    # Unifying the paragraphs
    list_paragraphs = []
    for p in np.arange(0, len(x)):
        paragraph = x[p].get_text()
        list_paragraphs.append(paragraph)
        final_article = " ".join(list_paragraphs)
        
    news_contents.append(final_article)

/article/6615311/Katoro-Gold-raises-capital-to-advance-Haneti-project-in-Tanzania.html
[]
/article/6615142/US-non-farm-employment-up-135k-in-February.html
[]
/article/6615079/Randall-Quilter-appoints-Mike-Walker-as-chief-restructuring-officer.html
[]
/article/6615068/Horizonte-Minerals-appoints-new-director-of-Brazil-nickel-project.html
[]
/article/6615065/Asos-appoints-four-new-directors-including-Ocado-executive.html
[]
/article/6614566/BATM-delivers-milestones-under-its-strategic-agreement-with-Arm-ahead-of-schedule.html
[]
/article/6614562/Metro-Bank-founder-and-chairman-Vernon-Hill-to-stand-down-by-year-end.html
[]
/article/6614552/Live-Company-to-stage-first-Bricklive-show-in-France-and-Thailand.html
[]
/article/6614550/Seeing-Machines-collaborates-with-Alaska-Airlines-on-tracking-technology.html
[]
/article/6614545/Trans-Siberian-Gold-hires-new-chief-engineer-for-Asacha-mine.html
[]
/article/6614539/Nu-Oil-and-Gas-overhauls-board-to-raise-capital.html
[]
/article/6614523/Cello-H

In [89]:
len(postedDate)


49

Let's put them into:
* a dataset which will the input of the models (`df_features`)
* a dataset with the title and the link (`df_show_info`)

In [90]:
# df_features
df_features = pd.DataFrame(
     {'Article Content': news_contents 
    })

# df_show_info
df_show_info = pd.DataFrame(
    {'Article Title': list_titles,
     'Article Link': list_links})

In [91]:
df=pd.DataFrame(columns= ["STOCK ID", "URL", "PUBLISH TIME", "EDITED TIME", "SCRAPED TIME", "TITLE", "CONTENT" ])
df['URL']=list_links
df['PUBLISH TIME']=postedDate
df['SCRAPED TIME']=scrapedTime
df['TITLE']=list_titles
df['CONTENT']=news_contents
df

,STOCK ID,URL,PUBLISH TIME,EDITED TIME,SCRAPED TIME,TITLE,CONTENT
0,NaN,/article/6615311/Katoro-Gold-raises-capital-to...,[],NaN,03/10/2019 01:17:07,\n\t\t\n\t\t\t\t\t\t\t\t\t\t\t\t02 Oct15:02\n\...,Tanzania-focused Katoro Gold said it had raise...
1,NaN,/article/6615142/US-non-farm-employment-up-135...,[],NaN,03/10/2019 01:17:07,\n\t\t\n\t\t\t\t\t\t\t\t\t\t\t\t02 Oct13:40\n\...,Tanzania-focused Katoro Gold said it had raise...
2,NaN,/article/6615079/Randall-Quilter-appoints-Mike...,[],NaN,03/10/2019 01:17:07,\n\t\t\n\t\t\t\t\t\t\t\t\t\t\t\t02 Oct13:32\n\...,Tanzania-focused Katoro Gold said it had raise...
3,NaN,/article/6615068/Horizonte-Minerals-appoints-n...,[],NaN,03/10/2019 01:17:07,\n\t\t\n\t\t\t\t\t\t\t\t\t\t\t\t02 Oct13:22\n\...,Tanzania-focused Katoro Gold said it had raise...
4,NaN,/article/6615065/Asos-appoints-four-new-direct...,[],NaN,03/10/2019 01:17:07,\n\t\t\n\t\t\t\t\t\t\t\t\t\t\t\t02 Oct13:20\n\...,Tanzania-focused Katoro Gold said it had raise...
5,NaN,/article/6614566/BATM-delivers-milestones-unde...,[],NaN,03/10/2019 01:17:07,\n\t\t\n\t\t\t\t\t\t\t\t\t\t\t\t02 Oct10:03\n\...,Tanzania-focused Katoro Gold said it had raise...
6,NaN,/article/6614562/Metro-Bank-founder-and-chairm...,[],NaN,03/10/2019 01:17:07,\n\t\t\n\t\t\t\t\t\t\t\t\t\t\t\t02 Oct10:01\n\...,Tanzania-focused Katoro Gold said it had raise...
7,NaN,/article/6614552/Live-Company-to-stage-first-B...,[],NaN,03/10/2019 01:17:07,\n\t\t\n\t\t\t\t\t\t\t\t\t\t\t\t02 Oct09:59\n\...,Tanzania-focused Katoro Gold said it had raise...
8,NaN,/article/6614550/Seeing-Machines-collaborates-...,[],NaN,03/10/2019 01:17:07,\n\t\t\n\t\t\t\t\t\t\t\t\t\t\t\t02 Oct09:57\n\...,Tanzania-focused Katoro Gold said it had raise...
9,NaN,/article/6614545/Trans-Siberian-Gold-hires-new...,[],NaN,03/10/2019 01:17:07,\n\t\t\n\t\t\t\t\t\t\t\t\t\t\t\t02 Oct09:53\n\...,Tanzania-focused Katoro Gold said it had raise...


In [92]:
st=pd.DataFrame(pd.read_csv('c:/data/CS/tickers_list.csv'))
st['Issuer Name'][0].split(',')
issuerName=[]
stockabbre=[]
for i in range(len(st['Issuer Name'])):
    issuername=st['Issuer Name'][i].split(',')
    issuerName.append(issuername[0])
print (issuerName)
stockabbre=st["Symbol"]
stockabbre=list(stockabbre)
stockabbre.append('APPL')
print (stockabbre)
    

['ADOMANI', 'AdvisorShares Trust', 'Allied Esports Entertainment', 'Alta Mesa Resources', 'Alta Mesa Resources', 'BIQI International Holdings Corporation', 'China Lending Corporation', 'China TechFaith Wireless Communication Technology Limited', 'ChinaCache International Holdings Ltd.', 'Cray Inc', "Del Frisco's Restaurant Group", 'Diversicare Healthcare Services Inc.', 'Emmaus Life Sciences', 'Emmaus Life Sciences', 'Finisar Corporation', "Fred's", 'Gladstone Capital Corporation', 'Hunter Maritime Acquisition Corp.', 'Hunter Maritime Acquisition Corp.', 'Hunter Maritime Acquisition Corp.', 'Ivy NextShares', 'Ivy NextShares', 'Ivy NextShares', 'Peak Resorts', 'Repay Holdings Corporation', 'Shutterfly', 'USA Technologies', 'USA Technologies', 'Xynomic Pharmaceuticals Holdings', 'Xynomic Pharmaceuticals Holdings', 'Xynomic Pharmaceuticals Holdings', 'Yangtze River Port and Logistics Limited ']
['ADOM', 'BKCH', 'AESEW', 'AMR', 'AMRWW', 'BIQI', 'CLDC', 'CNTF', 'CCIH', 'CRAY', 'DFRG', 'DVCR

In [96]:
for i in range(len(df['CONTENT'])):
    for j in range(len(st['Symbol'])):
        if st['Symbol'][j] in df['CONTENT'][i] or issuerName[j] in df['CONTENT']:
            df['STOCK ID'][i]=st['Symbol'][j]
df['STOCK ID'].describe()

count     0
unique    0
Name: STOCK ID, dtype: int64

In [68]:
df.to_csv("C:/data/CS/stockmarketwire.csv")